In [201]:
import numpy as np
import random


def get_tree(Grafs):
    n = len (Grafs)
    graf_tree = np.ndarray((n, n), dtype = np.int)
    for i in range(n):
        for j in range(i,n):
            if set(Grafs[i][1]) & set(Grafs[i][1]) != set():
                graf_tree[i, j] = graf_tree[j, i] = 1 
    #print(graf_tree)
    return graf_tree


def pick_span_tree(Graf_tree):
    n = len(Graf_tree)
    sp_tree_m = np.eye(n, dtype = np.int)
    
    av_edg = list()
    s = random.choice(range(n))
    for i in range(n):
        if (Graf_tree[s,i] == 1) and (s != i):
            av_edg.append((s,i))
    while len(av_edg) != 0:
        ch1, ch2 = random.choice(av_edg)
        tmp = list()
        for i in range(n):
            if (Graf_tree[ch2, i] == 1) and (ch2 != i):
                if (i, ch2) in av_edg:
                    av_edg.remove((i, ch2))
                else:
                    av_edg.append((ch2, i)) 
        sp_tree_m[ch1,ch2] = sp_tree_m[ch2, ch1] = 1
    #print(sp_tree_m)
    return sp_tree_m
        

def pick_travers(Sp_tree):
    n = len(Sp_tree)
    av = [random.choice(range(n))]
    trav = list()
    while len(av) != 0:
        ch = random.choice(av)
        av.remove(ch)
        trav.append(ch)
        for i in range(n):
            x = Sp_tree[ch,i]
            if (x == 1) and (i not in av) and (i not in trav):
                av.append(i)
    #print(trav)
    return trav
  

def DIST(A):
    n = len(A)
    D = np.ndarray((n, n))
    T = np.ndarray((n, n))
    B = np.matmul(A,A) / n
    for ind in range(n):           
        np.subtract(B, B[ind], out=T)
        np.fill_diagonal(T, 0)
        D[ind] = np.amax(np.abs(T), axis=1)
    return D.T 


def DE2(graf1, graf2):
    D_A, ind_a = graf1
    D_B, ind_b = graf2
    a_set, b_set = set(ind_a), set(ind_b)
    n = len(a_set | b_set)
    D = np.ndarray((n, n))

    
    blok_1_ind = sorted(a_set - b_set)
    ind = list(map(ind_a.get, blok_1_ind))
    tmp_a = D_A.take(ind, 0)
    blok_1 = tmp_a.take(ind, 1)
    #print('blok_1\n',blok_1)
    
    blok_9_ind = sorted(b_set - a_set)
    ind = list(map(ind_b.get, blok_9_ind))
    tmp_b = D_B.take(ind, 0)
    blok_9 = tmp_b.take(ind, 1)
    #print('blok_9\n',blok_9)
    
    blok_5_ind = sorted(b_set & a_set)
    ind_1, ind_2 = list(map(ind_a.get, blok_5_ind)),  list(map(ind_b.get, blok_5_ind))
    t1 = D_A.take(ind_1, 1).take(ind_1, 0) 
    t2 = D_B.take(ind_2, 1).take(ind_2, 0)
    blok_5 = (t1 + t2)/2
    #print('blok_5\n',blok_5)
     
    i_n, j_n = len(b_set - a_set), len(a_set - b_set), 
    blok_2_4 = tmp_a.take(ind_1, 1)
    blok_6_8 = tmp_b.take(ind_2, 1)
    #print('blok_2_4\n',blok_2_4)
    #print('blok_6_8\n',blok_6_8)
 
    blok_3_7 = np.ndarray((i_n, j_n))
    Ta, Ts1, Ts2 = np.ndarray(blok_2_4.shape), np.ndarray(blok_2_4.shape), np.ndarray(blok_2_4.shape)
    for i in range(i_n):
        np.add(blok_2_4, blok_6_8[i], out=Ta)
        np.subtract(blok_2_4, blok_6_8[i], out=Ts1)
        np.fabs(Ts1, out=Ts2)
        t1, t2 = np.amin(Ta, axis=1), np.amax(Ts2, axis=1)    
        t3 = np.where(t1 + t2 == 0, 0, 1/(t1+t2))        
        blok_3_7[i] = 2*(t1*t2)/t3
    #print('blok_3_7\n',blok_3_7)
            
    D = np.block([[blok_1, blok_2_4, blok_3_7.T], [blok_2_4.T, blok_5, blok_6_8.T], [blok_3_7, blok_6_8, blok_9]])
           
    d_ind = dict(zip(blok_1_ind + blok_5_ind + blok_9_ind, range(n)))
           
    return (D, d_ind)   


def DE(Grafs, trav):
    visited = list()
    D_graf = Grafs[trav[0]]
    for t in trav[1:]:
        if t in visited:
            continue
        visited.append(t) 
        D_graf = DE2(D_graf, Grafs[t]) 
    #print(D_graf)
    return D_graf


def NBSE0(A, Grafs, I, J):
    D = None
    graf_tree = get_tree(Grafs)
    for i in range(I):        
        sp_tree = pick_span_tree(graf_tree)
        for j in range(J):
            trav = pick_travers(sp_tree)
            if D is None:
                D, d = DE(Grafs, trav)
            else: 
                D += DE(Grafs, trav)[0]      
    D = D / (I*J)
    
    t = {v: k for k, v in d.items()}
    ind = list(map(t.get, list(range(len(D)))))
    A_per = A.take(ind, axis=0).take(ind, axis=1)
    
    P, N = NBS(A_per, D)
    return (P, A_per, N), d
    

def NBS(A, D):
    n = len(D)
    tr = int(np.sqrt(np.log(n)/n)*n) 
    N = np.argsort(D, axis=1).take(list(range(1,tr+1)), axis=1)
    P = np.ndarray((n,n))    
    for i in range(n):
        #print(np.sum(A.take(N[i], axis=1),axis=1),D.take(N[i], axis=1)[i] )
        P[i] = np.sum(A.take(N[i], axis=1), axis=1) / tr
    np.fill_diagonal(P, 1)
    P = (P + P.T)/2
    return P,N


def CORR(P, A, N):
    n = len(A)
    tr = N.shape[1]
    P_cor = np.ndarray((n,n))
    A_cor = np.where(A == 0 , P, 1) 
    for i in range(n):
        P_cor[i] = np.sum(A_cor.take(N[i], axis=1), axis=1) / tr
    np.fill_diagonal(P_cor, 1)
    P_cor = (P_cor + P_cor.T)/2
    return ( P_cor, A, N)


#A - affinity matrix
#Grafs - list of tuples that contain  part of  affinity matrix
#and dictionary which mapping vertex number to corresponding row in given matrix  
#delta - threshold for smoothing algorithm
def NBSE(A, Grafs, delta, I=2, J=1):    
    n_res,d = NBSE0(A, Grafs, I, J)
    n = len(n_res[0])
    while True:
        o_res = n_res
        n_res = CORR(*n_res)
        diff = n_res[0] - o_res[0]
        absdiff = np.square(diff)
        diff_sum = np.sqrt(np.sum(absdiff))/n**2
        if diff_sum < delta:
            break
    return n_res[0],d

SyntaxError: invalid syntax (<ipython-input-201-2255182790ab>, line 63)

In [169]:
def sample_grafon(grafon, n):
    S = rng.random(size=(n,))
    V = np.fromiter((grafon(S[i], S[j]) for i in range(n-1) for j in range(i+1, n)), count=int((n)*(n-1)/2), dtype=np.float)
    def sample(x):
        global rng
        if rng.random() < x:
            return 1
        else:
            return 0
        
    uf = np.frompyfunc(sample, 1, 1)
    P = uf(V)
    
    R = np.eye(n, dtype=np.int)
    for i in range(n-1):
        for j in range(i+1,n):
            R[i,j] = R[j,i] = P[int(i*n - (i+1)*(i+2)/2 + j)]         
    P = np.eye(n)
    for i in range(n-1):
        for j in range(i+1,n):
            P[i,j] = P[j,i] = V[int(i*n - (i+1)*(i+2)/2 + j)]
    
    return R,P


def sort_graf_by_weight(A):
    ret = np.add.reduce(A)
    s = np.argsort(ret)
    gr = np.take(A, s, axis=1)
    gr = np.take(gr, s, axis=0)
    return gr
    
def d_gr(grafon, gr):
    n = len(gr)
    diff = 0
    for i in range(n):
        for j in range(n):
            diff += np.absolute(grafon((i+0.5)/n,(j+0.5)/n)- gr[i,j])/n**2
            #print(grafon((i+0.5)/n,(j+0.5)/n)- gr[i,j])
    return diff

def square_er(P,P_e):
    q = np.sum((P-P_e)**2)/(P_e.shape[1]*P.shape[0])
    return np.sqrt(q) 

In [170]:
P,h = NBS(A,DA)

[21. 16. 14. 24. 12. 21. 19.  1.  0. 24. 17. 20.  3. 21. 21.  0. 14. 15.
 18.  2.  1.  6. 28.  3. 24. 13.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0. 46.  0. 13.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  1.  2.  0.  0.  0.  2.  0.  0. 28. 15. 24. 13. 12. 14. 21.  0.
 15.  3.  0. 12. 11.  8.  1.  9. 21.  2.  3.  2. 13. 12. 26. 15.  5. 25.
 26. 14.  1.  5.  4. 12. 51. 12. 17. 18. 23. 19. 17.  1. 22.  0.  6. 19.
 32.  5.  0.  0. 20.  6.  9.  4.  2.  3.  1.  3.  3.  9.  2. 10.  3.  9.
  1.  1.  0.  0.  0.  6. 24. 14.  2.  9.  5.  1.  2.  2.  7.  2. 16.  2.
  2.  2.  0.  4.  2.  1.  1.  6.  2. 16. 22.  0.  0.  1.  0.  0.  1.  0.
  0.  0.  1.  0.  0.  0.  5. 15. 12. 23. 20.  7. 34. 30. 33.  1.  4. 11.
 43.  0. 24. 13.  5. 20. 36.  5. 52. 14.  4.  9.  3.  7.  5.  1. 52. 17.
  1. 15. 12.  9.  0. 10.  2. 40. 19.  6.  5.  8. 16. 26. 24. 13.  5. 16.
 56.  2.  3. 46.  0.  0. 20. 47. 32.  4.  0.  0.  0

  1.  1.  1.] [0.04441156 0.04441156 0.04441156 0.04441156 0.04441156 0.04441156
 0.04441156 0.04441156 0.04441156 0.04441156 0.04441156 0.04441156
 0.04441156 0.04441156 0.06280743 0.06280743 0.06280743 0.06280743
 0.06280743 0.06280743 0.06280743 0.06280743 0.06280743 0.06280743
 0.06280743 0.06280743 0.06280743 0.06280743 0.06280743 0.06280743
 0.06280743 0.06280743 0.06280743 0.06280743 0.06280743 0.06280743
 0.06280743 0.06280743 0.06280743 0.06280743 0.06280743 0.06280743
 0.06280743 0.06280743 0.06280743 0.06280743 0.06280743 0.06280743
 0.06280743 0.06280743 0.06280743 0.06280743 0.06280743 0.06280743
 0.06280743 0.06280743]
[ 2.  1.  7.  4.  7.  2.  7.  0.  1. 15.  3.  3.  1.  8.  4.  0. 10. 11.
 21. 10.  3. 15. 29. 10. 22. 12.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0. 54.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  3.  3.  3.  0.  9.  0.  1. 23. 13. 18.  3.  0.  0.  4

[ 0.  9.  3.  6.  4.  5.  6. 24.  3. 19.  3.  9.  1.  5.  4.  1. 30. 10.
 31.  4.  3.  6. 11.  4. 34.  6.  3.  3.  3.  2.  3.  2.  3.  4.  5.  2.
  2.  3.  2.  4.  5.  7.  2.  2.  3.  3.  2.  2.  2.  2.  3.  2.  2.  2.
  2.  3.  3.  5.  2.  2. 38.  1.  0.  0.  0.  0.  0.  1.  0.  0.  1.  2.
  2.  2.  0.  2.  5.  5.  5.  8. 21.  6. 21. 16. 17.  8.  9.  9.  9.  9.
  9.  9.  9.  9.  9.  9.  8.  9. 10.  9.  8.  8.  9.  8.  6. 10.  9.  9.
 25.  8.  3.  3.  3. 18. 37.  8.  9.  3.  2.  5.  3.  8.  9.  1.  8.  9.
 23.  7.  5.  4. 10.  5.  5.  6. 25. 10.  3.  6.  2. 21.  3. 21. 10.  5.
  3.  4.  1.  1.  1.  7. 27. 10. 10.  5. 13.  2.  5.  2.  8. 16. 34.  8.
 22. 26. 21. 34. 22. 19. 25.  6. 26. 33.  5. 11.  6.  2.  9. 20. 10. 10.
 10. 12. 28. 26.  7.  9.  3. 11.  6. 19. 13. 25. 39.  7. 23. 12. 14. 24.
 26.  7. 14. 16.  8. 22. 21.  2. 30. 19.  3. 12.  3.  1.  2.  1. 35. 23.
  1. 11.  8. 30. 18.  5.  2. 37.  7.  4.  4.  4.  9. 12. 11.  9.  6. 40.
 46. 30. 28. 44.  8.  3. 25. 44. 29.  6.  4.  6.  4

[ 0.  9.  3.  6.  4.  5.  6. 25.  3. 22.  3.  9.  1.  5.  4.  1. 32. 10.
 32.  4.  3.  6. 11.  4. 36.  6.  3.  3.  4.  2.  4.  2.  3.  4.  6.  2.
  2.  3.  2.  4.  5.  7.  2.  2.  3.  3.  2.  2.  2.  2.  3.  2.  2.  2.
  2.  2.  3.  5.  2.  2. 40.  1.  0.  0.  0.  0.  0.  1.  0.  0.  1.  2.
  2.  2.  0.  2.  5.  5.  5.  9. 23.  6. 24. 19. 20.  8.  9.  9.  9.  9.
  9.  9.  9.  9.  9.  9.  8.  9. 10.  9.  8.  8.  9.  8.  6. 10.  9.  9.
 27.  8.  3.  3.  3. 20. 39.  8.  9.  3.  2.  5.  3.  8.  9.  1.  8.  9.
 25.  9.  5.  4. 10.  5.  5.  6. 27. 13.  3.  6.  2. 23.  3. 24. 12.  5.
  3.  4.  1.  1.  1.  7. 29. 10. 10.  5. 15.  2.  5.  2.  8. 18. 36. 10.
 24. 29. 23. 36. 22. 21. 26.  6. 28. 35.  5. 11.  6.  2.  9. 23. 10. 10.
 10. 12. 30. 27.  7.  9.  3. 11.  6. 22. 13. 27. 41.  7. 26. 13. 15. 26.
 29.  7. 14. 18.  8. 25. 24.  2. 32. 22.  3. 13.  3.  1.  2.  1. 37. 25.
  1. 11. 10. 32. 18.  5.  2. 39.  7.  4.  4.  4.  9. 12. 11.  9.  6. 42.
 48. 32. 30. 46.  8.  3. 28. 46. 31.  6.  4.  6.  4

  0.  0.  0.] [0.14044168 0.16012815 0.16617284 0.16617284 0.16617284 0.17200523
 0.17200523 0.17200523 0.17764624 0.17764624 0.17764624 0.17764624
 0.17764624 0.17764624 0.17764624 0.17764624 0.17764624 0.18311355
 0.18311355 0.18311355 0.18311355 0.18311355 0.18311355 0.18311355
 0.18842229 0.18842229 0.18842229 0.18842229 0.18842229 0.18842229
 0.18842229 0.18842229 0.18842229 0.18842229 0.18842229 0.18842229
 0.1935855  0.1935855  0.1935855  0.1935855  0.1935855  0.1935855
 0.1935855  0.1935855  0.1935855  0.1935855  0.1935855  0.1935855
 0.1935855  0.1935855  0.1935855  0.1935855  0.1935855  0.1935855
 0.1935855  0.19861453]
[ 2.  1.  2.  3.  3.  3.  1.  0.  0. 13.  3.  2.  1.  3.  4.  0.  6.  4.
 12.  1.  0.  1. 27.  0. 15.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1. 53.  0.  1.  0.  1.  0.  0.  1.  0.  0.  0.  0.
  1.  0.  1.  1.  2.  2.  1.  4.  3.  2. 17. 11. 12.  1.  1.  1.  3.  

...

 0.32936494 0.32936494]
[ 1.  3.  3.  3.  3.  2.  5.  0.  0. 16.  1.  3.  1.  3.  2.  0.  8.  6.
 14.  2.  0.  5. 23.  3. 13.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0. 52.  0.  3.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  1.  2.  2.  5.  2.  7.  0.  2. 20. 13. 16.  2.  1.  1.  3.  1.
  1.  2.  1.  3.  2.  1.  1.  1.  3.  1.  1.  2.  5.  3.  8.  1.  1.  6.
 28.  2.  1.  2.  1. 14. 51.  2.  4.  7.  4.  4.  7.  1.  2.  0.  1.  3.
 36.  7.  0.  0. 22. 10.  9.  5.  5.  4.  1.  7.  5. 16.  3.  5.  8.  5.
  1.  1.  0.  0.  0.  5. 25.  5.  3.  3.  2.  1.  3.  3.  8.  4. 27.  9.
  4.  4.  0. 12.  2.  1.  2.  7.  7. 21. 16.  1.  2.  3.  1.  1.  4.  0.
  1.  5.  1.  0.  2.  0.  3. 25. 13. 25. 27.  6. 25.  9. 38.  2.  5. 10.
 46.  0. 22. 10.  6. 25. 23.  6. 53. 17.  5.  9.  4.  7.  6.  2. 47.  6.
  0.  4.  7. 13.  5.  9.  4. 23. 25.  6.  2.  5.  6. 11.  4.  5.  3. 34.
 55.  2.  8. 39.  0.  1. 31

[ 0.  0.  0.  1.  1.  0.  1.  1.  0. 13.  0.  0.  0.  0.  0.  0.  2.  3.
 21.  4.  0.  2. 40.  1. 19.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0. 55.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  2.  1.  2.  2.  5.  1.  1. 14. 11. 11.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  2.  0.  0.  2.
 35.  0.  1.  0.  0.  9. 55.  0.  1.  6.  0.  0.  2.  0.  0.  0.  0.  0.
 45.  4.  0.  0. 18. 10.  6.  7.  4.  7.  0. 15.  4. 21.  0.  8.  4.  6.
  0.  0.  0.  0.  0.  9. 26.  5.  3.  1.  6.  0.  8.  1. 11. 11. 50. 21.
  8.  8.  3. 24.  3.  2.  1. 17. 17. 28. 19.  5.  2.  4.  6.  4.  8.  0.
  5.  4.  4.  3.  4.  2.  1. 15. 11. 26. 34. 13. 15. 12. 43.  1.  2.  7.
 49.  0. 18.  4.  1. 23. 18.  0. 55. 11.  1.  2.  0.  0.  0.  0. 44.  5.
  0.  2.  8. 18.  9.  1.  0. 21. 26.  5.  0.  4.  1.  9.  0.  5.  0. 49.
 56.  6.  9. 46.  0.  0. 40. 50. 21.  8.  0.  0.  0

[ 0.  9.  3.  6.  4.  5.  6. 22.  3. 19.  3.  9.  1.  5.  4.  1. 30. 10.
 29.  4.  3.  6. 10.  4. 34.  6.  3.  3.  3.  2.  3.  1.  3.  4.  6.  2.
  2.  3.  2.  4.  5.  7.  1.  2.  3.  3.  2.  2.  2.  2.  3.  2.  2.  2.
  2.  3.  3.  5.  2.  1. 39.  1.  0.  1.  0.  1.  0.  1.  0.  0.  1.  2.
  2.  2.  0.  2.  5.  5.  5.  9. 20.  6. 21. 16. 17.  8.  9.  9.  9.  9.
  9.  9.  9.  9.  9.  9.  8.  9. 10.  9.  8.  8.  9.  8.  6. 10.  9.  9.
 26.  8.  3.  3.  3. 17. 38.  8.  9.  3.  2.  5.  3.  8.  9.  1.  8.  9.
 24.  8.  5.  4.  9.  5.  5.  5. 25. 10.  3.  5.  2. 22.  3. 19. 11.  5.
  3.  4.  1.  1.  1.  6. 27. 10. 10.  5. 13.  2.  4.  2.  7. 16. 35.  8.
 23. 28. 21. 35. 21. 18. 25.  5. 24. 32.  5. 11.  6.  2.  9. 20. 10.  9.
  9. 12. 28. 25.  7.  8.  3. 11.  6. 19. 13. 27. 39.  7. 24. 12. 15. 24.
 24.  7. 13. 17.  8. 22. 21.  2. 31. 19.  3. 13.  3.  1.  2.  1. 33. 23.
  1. 11.  8. 31. 18.  5.  2. 37.  7.  4.  4.  4.  9. 12. 11.  9.  6. 41.
 47. 32. 30. 45.  8.  3. 24. 44. 29.  5.  4.  6.  4

[ 7.  5.  4.  6.  3.  5.  7.  7.  0. 46.  3.  8.  1. 10.  6.  0. 36. 17.
 44. 11.  1. 11. 40.  7. 18. 20.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  3. 50.  1.  3.  2.  6.  2.  9.  0.  8.  8.  1.  0.
  2.  8.  2. 18.  1.  4.  1. 36.  1.  0. 50. 43. 44.  5.  5.  4.  8.  3.
 10.  3.  3.  8.  6.  5.  4.  6.  7.  3.  3.  5.  7.  4. 15. 11.  7.  9.
 52.  5.  3.  2.  2. 43. 54.  7.  7.  8.  9.  9.  9.  4.  8.  4.  4.  7.
 53. 37.  1.  1. 22. 13.  8.  6. 11. 16.  1.  4.  5. 14.  7. 15. 16.  7.
  1.  1.  0.  0.  0.  9. 47. 21. 12. 12.  9.  1.  6.  3. 10. 13. 24. 10.
  8.  2.  1. 10.  0.  1.  1. 19.  6. 27. 14.  1.  3.  4.  1.  3.  1.  0.
  0.  3.  1.  2.  3.  1. 11. 43. 17. 52. 52. 10. 51. 20. 53.  8. 37. 48.
 41.  5. 48. 40.  8. 53. 54.  3. 53. 47.  6. 40.  6.  4.  4.  3. 46. 19.
  0. 19. 24. 38.  6. 10.  8. 51. 22. 13.  5.  9. 18. 35. 32. 23.  6. 32.
 54.  2.  4. 53.  2.  6. 46. 50. 35.  4.  3.  2.  1

[ 0.  8.  3.  6.  4.  4.  6. 24.  3. 19.  3.  8.  1.  5.  4.  1. 30. 10.
 31.  4.  3.  6. 10.  4. 34.  6.  4.  4.  4.  3.  3.  2.  3.  4.  5.  1.
  1.  2.  1.  3.  4.  6.  2.  2.  3.  3.  2.  2.  1.  2.  3.  2.  2.  2.
  2.  3.  3.  4.  2.  2. 37.  1.  0.  0.  0.  0.  0.  1.  0.  0.  1.  2.
  2.  2.  0.  2.  5.  5.  5.  8. 22.  6. 21. 17. 18.  7.  8.  8.  8.  8.
  8.  8.  8.  8.  8.  8.  7.  8.  9.  8.  7.  7.  8.  7.  6.  9.  8.  8.
 26.  7.  3.  3.  3. 20. 37.  7.  8.  3.  2.  5.  3.  7.  8.  1.  7.  8.
 24.  9.  5.  4.  9.  5.  5.  6. 26. 13.  3.  6.  2. 23.  3. 23. 12.  5.
  3.  4.  1.  1.  1.  7. 27.  9.  9.  5. 15.  2.  5.  2.  7. 16. 35. 12.
 22. 26. 21. 34. 18. 18. 25.  6. 26. 32.  5. 10.  6.  2.  9. 20.  9.  9.
  9. 10. 27. 23.  7.  8.  3. 10.  6. 19. 12. 25. 38.  6. 23. 14. 13. 25.
 27.  7. 13. 17.  7. 22. 22.  2. 31. 20.  3. 11.  3.  1.  2.  1. 35. 23.
  1. 10.  8. 29. 19.  5.  2. 36.  7.  4.  4.  4.  8. 11. 10.  8.  5. 39.
 45. 30. 28. 43.  8.  3. 26. 43. 31.  6.  4.  5.  3

In [164]:
with open("econ-beaflw.mtx") as f:
    n = 0
    for x in f:
        if x[0] == '%':
            continue
        line = x.split()
        lm = max(int(line[0]),int(line[1]))
        if lm > n:
            n = lm
            
A = np.ndarray((n,n))
np.fill_diagonal(A, 1)
with open("econ-beaflw.mtx") as f:
    for s in f:
        if s[0] == '%':
            continue
        line = s.split()
        x, y = int(line[0]) - 1 , int(line[1]) - 1
        if abs(float(line[2])) != 0:
            A[x,y] = 1
rng = np.random.default_rng()

k = 1
t = 2
m = 0.3
grafs = list()
for x in range(t):
    
    l = int(n*(m+1)/t)
    v = rng.choice(n, l, replace=False)
    v.sort()
    d = dict(zip(v, range(l)))
     
    T = A.take(v, axis=0).take(v, axis=1)
    
    T = np.where(rng.random((l,l)) < k , T, 0)
    grafs.append((DIST(T), d))

d = dict()
for g in grafs:
    d.update(g[1])
M = A.copy()
ind = set(d)
for i in set(range(n)) - ind:
    M[:,i] = M[i] = 0

DA, DM = DIST(A), DIST(M)

P,h = NBS(A,DA)

P_e,d = NBSE(M,grafs,1,4,4)
p_e = P_e[0]
t = {v: k for k, v in d.items()}
ind = list(map(t.get, range(len(P_e))))
P_pre = P.take(ind, axis=0).take(ind, axis=1)

square_er(P_pre, P_e)

[[  0  90  88 ... 467 455 496]
 [  1  93 105 ... 445 455 496]
 [  2   4  90 ... 467 455 496]
 ...
 [504 506  48 ... 467 455 496]
 [505 484 485 ... 467 455 496]
 [506  47  49 ... 467 455 496]] [[ 90  88 116 ... 212 226 217]
 [ 93 105  94 ...   6   3 437]
 [  4  90 102 ... 120 428 156]
 ...
 [506  48 484 ...  32  33  34]
 [484 485 487 ...  32  33  34]
 [ 47  49 484 ...  32  33  34]] [[0.14729648 0.16012815 0.16012815 ... 0.17764624 0.14044168 0.21299036]
 [0.11750194 0.12561486 0.12561486 ... 0.1935855  0.1935855  0.1935855 ]
 [0.16617284 0.17200523 0.17764624 ... 0.18842229 0.19861453 0.17200523]
 ...
 [0.26274233 0.27014497 0.25512498 ... 0.30769231 0.30769231 0.28437273]
 [0.26274233 0.27014497 0.25512498 ... 0.30769231 0.30769231 0.28437273]
 [0.26274233 0.27014497 0.25512498 ... 0.30769231 0.30769231 0.28437273]]
[[  0 352 347 ... 368 377 427]
 [  1 380 354 ... 419 417 427]
 [  2 390 331 ... 417 419 427]
 ...
 [431 102   6 ...  90  48  98]
 [432 103 102 ...  90  48  98]
 [433 102 10

0.13089101612209494

In [200]:
grafon = lambda x,y :np.sin(x/2+y)#1- 0.5*max(x,y)
n = 100
sam_gr,P = sample_grafon(grafon, n)

k = 1
t = 4
m = 0.3
grafs = list()
for x in range(t):
    
    l = int(n*(m+1)/t)
    v = rng.choice(n, l, replace=False)
    v.sort()
    d = dict(zip(v, range(l)))
     
    T = sam_gr.take(v, axis=0).take(v, axis=1)
    
    T = np.where(rng.random((l,l)) < k , T, 0)
    grafs.append((DIST(T), d))

sam_gr = np.where(rng.random((n,n)) < k , sam_gr, 0)
    
D = DIST(sam_gr)

#d = {i:i for i in range(n)}
#P_e,N =  NBSE(sam_gr,[(D,d)],0.1, I=1,J=1)

P_e,N = NBS(sam_gr,D)
#P_e,N = NBS(sam_gr,DIST(P_e))
#P_e,N = NBS(P_e,DIST(P_e))

P_ne,d = NBSE(sam_gr,grafs,0.1, I=6,J=6)


t = {v: k for k, v in d.items()}
ind = list(map(t.get, range(len(P_ne))))
P_pre = P.take(ind, axis=0).take(ind, axis=1)

print(P,'\n\n',P_e,'\n\n',P_ne)
print(square_er(P,P_e))
print(square_er(P_pre,P_ne))

[[1.         0.9580569  0.94639471 ... 0.77735776 0.41783248 0.95233611]
 [0.9580569  1.         0.97831884 ... 0.84723109 0.52384176 0.98206177]
 [0.94639471 0.97831884 1.         ... 0.83691499 0.50745266 0.9782754 ]
 ...
 [0.77735776 0.84723109 0.83691499 ... 1.         0.34900627 0.92697131]
 [0.41783248 0.52384176 0.50745266 ... 0.34900627 1.         0.8172131 ]
 [0.95233611 0.98206177 0.9782754  ... 0.92697131 0.8172131  1.        ]] 

 [[1.         0.88095238 0.73809524 ... 0.52380952 0.33333333 0.97619048]
 [0.88095238 1.         0.97619048 ... 0.83333333 0.71428571 1.        ]
 [0.73809524 0.97619048 1.         ... 0.61904762 0.54761905 0.9047619 ]
 ...
 [0.52380952 0.83333333 0.61904762 ... 1.         0.33333333 0.85714286]
 [0.33333333 0.71428571 0.54761905 ... 0.33333333 1.         0.83333333]
 [0.97619048 1.         0.9047619  ... 0.85714286 0.83333333 1.        ]] 

 [[1.         0.95447531 0.96141975 ... 0.98611111 0.8433642  0.93441358]
 [0.95447531 1.         0.9652777